# Instant Gratification!

Let's build a useful LLM solution - in a matter of minutes.

Our goal is to code a new kind of Web Browser. Give it a URL, and it will respond with a summary. The Reader's Digest of the internet!!

Before starting, be sure to have followed the instructions in the "README" file, including creating your API key with OpenAI and adding it to the `.env` file.

## If you're new to Jupyer Lab

Welcome to the wonderful world of Data Science experimentation! Once you've used Jupyter Lab, you'll wonder how you ever lived without it. Simply click in each "cell" with code in it, like the cell immediately below this text, and hit Shift+Return to execute that cell. As you wish, you can add a cell with the + button in the toolbar, and print values of variables, or try out variations.

If you need to start again, go to Kernel menu >> Restart kernel.

In [1]:
# imports

import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

# Connecting to OpenAI

The next cell is where we load in the environment variables in your `.env` file and connect to OpenAI.

## Troubleshooting if you have problems:

1. OpenAI takes a few minutes to register after you set up an account. If you receive an error about being over quota, try waiting a few minutes and try again.
2. Also, double check you have the right kind of API token with the right permissions. You should find it on [this webpage](https://platform.openai.com/api-keys) and it should show with Permissions of "All". If not, try creating another key by:
- Pressing "Create new secret key" on the top right
- Select **Owned by:** you, **Project:** Default project, **Permissions:** All
- Click Create secret key, and use that new key in the code and the `.env` file (it might take a few minutes to activate)
- Do a Kernel >> Restart kernel, and execute the cells in this Jupyter lab starting at the top
4. As a fallback, replace the line `openai = OpenAI()` with `openai = OpenAI(api_key="your-key-here")` - while it's not recommended to hard code tokens in Jupyter lab, because then you can't share your lab with others, it's a workaround for now
5. Contact me! Message me or email ed@edwarddonner.com and we will get this to work.

Any concerns about API costs? See my notes in the README - costs should be minimal, and you can control it at every point.

In [11]:
# Load environment variables in a file called .env

load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY','your-key-if-not-using-env')
openai = OpenAI()

In [37]:
# A class to represent a Webpage

class Website:
    url: str
    title: str
    text: str

    def __init__(self, url):
        self.url = url
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [38]:
# Let's try one out

ed = Website("https://edwarddonner.com")
print(ed.title)
print(ed.text)

Home - Edward Donner
Home
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI startup untapt,
acquired in 2021
.
We work with groundbreaking, proprietary LLMs verticalized for talent, we’ve
patented
our matching model, and our award-winning platform has happy customers and tons of press coverage.
Connect
with me for

## Types of prompts

You may know this already - but if not, you will get very familiar with it!

Models like GPT4o have been trained to receive instructions in a particular way.

They expect to receive:

**A system prompt** that tells them what task they are performing and what tone they should use

**A user prompt** -- the conversation starter that they should reply to

In [39]:
system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [40]:
def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "The contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

## Messages

The API from OpenAI expects to receive messages in a particular structure.
Many of the other APIs share this structure:

```
[
    {"role": "system", "content": "system message goes here"},
    {"role": "user", "content": "user message goes here"}
]

In [41]:
def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

## Time to bring it together - the API for OpenAI is very simple!

In [42]:
def summarize(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = messages_for(website)
    )
    return response.choices[0].message.content

In [43]:
summarize("https://edwarddonner.com")

'# Summary of Edward Donner\'s Website\n\nThe website is a personal platform for Ed, who shares his interests in coding, experimenting with large language models (LLMs), DJing, and electronic music production. He is the co-founder and CTO of [Nebula.io](http://nebula.io), a company focused on using AI for talent management. Ed has previous experience as the founder and CEO of the AI startup untapt, which was acquired in 2021.\n\n## Key Features:\n- **Outsmart LLM Arena**: A feature that showcases LLMs competing in scenarios focusing on diplomacy and strategy.\n- **Resources**: Ed offers posts related to selecting and fine-tuning LLMs for various applications.\n\n## Recent Posts:\n- **August 6, 2024**: "Outsmart LLM Arena – a battle of diplomacy and deviousness"\n- **June 26, 2024**: "Choosing the Right LLM: Toolkit and Resources"\n- **February 7, 2024**: "Fine-tuning an LLM on your texts: a simulation of you"\n- **January 31, 2024**: "Fine-tuning an LLM on your texts: part 4 – QLoRA"\n

In [44]:
def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [45]:
display_summary("https://edwarddonner.com")

# Summary of Edward Donner's Website

The website belongs to Ed Donner, a co-founder and CTO at Nebula.io, which focuses on using AI to help individuals discover their potential and optimize talent recruitment processes. Ed expresses his passion for coding and experimenting with large language models (LLMs), as well as his interests in DJing and electronic music production.

## Recent Posts
1. **August 6, 2024** - **Outsmart LLM Arena**: Discusses a platform where LLMs compete in diplomacy and strategy.
2. **June 26, 2024** - **Choosing the Right LLM**: Provides a toolkit and resources for selecting appropriate LLMs.
3. **February 7, 2024** - **Fine-tuning an LLM on Your Texts**: Offers insights on personalizing LLMs using individual text samples.
4. **January 31, 2024** - **Fine-tuning an LLM on Your Texts: Part 4 – QLoRA**: Continuation of the fine-tuning discussion focusing on a specific method. 

Ed invites engagement and connection through various platforms.

In [46]:
display_summary("https://cnn.com")

# Summary of CNN Website

The CNN website serves as a comprehensive news platform that provides a variety of content including breaking news articles, in-depth analysis, and videos across multiple categories such as world news, politics, business, health, entertainment, science, and more.

### Current Highlights:
- **International Conflicts**: Ongoing updates on the Israel-Hamas war and the Ukraine-Russia conflict, including Israel's military actions in response to Iranian missile attacks.
- **Politics**: Coverage of the 2024 U.S. elections, recent vice presidential debate highlights, and discussions around election certification challenges.
- **Health**: Emphasis on the importance of flu and COVID-19 vaccinations in the wake of seasonal outbreaks.
- **Tragedies and Accidents**: Reports on a school bus fire in Thailand that resulted in multiple fatalities and severe impacts from recent weather events in the U.S.
- **Cultural Events**: Insights into significant events including the opening of the world’s first Nintendo Museum and highlights from Paris Fashion Week.

### Additional Features:
- **Video Content**: The site includes video segments featuring news updates, expert opinions, and live television options.
- **Social Issues**: Articles covering societal impacts such as press freedom, AI regulation, and community health initiatives.

In summary, CNN remains a key source for immediate news and diverse storytelling, with a strong focus on global events and pressing issues affecting society.

In [47]:
display_summary("https://anthropic.com")

# Anthropic Overview

Anthropic is an AI safety and research company based in San Francisco, focused on developing AI systems that prioritize safety and integrity. The organization emphasizes interdisciplinary collaboration among experts in machine learning, physics, policy, and product development.

## Key Features

- **Claude**: A series of advanced AI models, with the latest being **Claude 3.5 Sonnet**, advertised as the most intelligent version yet.
- **API**: Allows developers to integrate and build applications using Claude.
  
## Recent Announcements

- **Claude 3.5 Sonnet Launch** (Jun 21, 2024): Introduction of their latest AI model, enhancing intelligence and capabilities.
- **Research on AI Safety**:
  - **Constitutional AI: Harmlessness from AI Feedback** (Dec 15, 2022): Focused on ensuring AI models operate safely.
  - **Core Views on AI Safety** (Mar 8, 2023): Discusses the principles and rationale behind AI safety practices.

Anthropic is committed to researching and producing reliable AI systems while fostering a safe and beneficial environment for AI development.

## An extra exercise for those who enjoy web scraping

You may notice that if you try `display_summary("https://openai.com")` - it doesn't work! That's because OpenAI has a fancy website that uses Javascript. There are many ways around this that some of you might be familiar with. For example, Selenium is a hugely popular framework that runs a browser behind the scenes, renders the page, and allows you to query it. If you have experience with Selenium, Playwright or similar, then feel free to improve the Website class to use them. Please push your code afterwards so I can share it with other students!

In [48]:
display_summary("https://openai.com")

# Website Summary

The website appears to be currently inaccessible due to requirements for JavaScript and Cookies to be enabled in the user's browser. Consequently, there are no visible contents or announcements available for summary. Users are prompted to adjust their browser settings to gain access to the website's features.

In [62]:
#Parse webpages which is designed using JavaScript heavely
# download the chorme driver from here as per your version of chrome - https://developer.chrome.com/docs/chromedriver/downloads
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options

PATH_TO_CHROME_DRIVER = '..\\path\\to\\chromedriver.exe'

class Website:
    url: str
    title: str
    text: str
    
    def __init__(self, url):
        self.url = url

        options = Options()

        options.add_argument("--no-sandbox")
        options.add_argument("--disable-dev-shm-usage")

        service = Service(PATH_TO_CHROME_DRIVER)
        driver = webdriver.Chrome(service=service, options=options)
        driver.get(url)

        input("Please complete the verification in the browser and press Enter to continue...")
        page_source = driver.page_source
        driver.quit()

        soup = BeautifulSoup(page_source, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.get_text(separator="\n", strip=True)

In [63]:
display_summary("https://openai.com")

# OpenAI Summary

OpenAI is focused on creating safe Artificial General Intelligence (AGI) that benefits all of humanity. The website features various AI products and research initiatives, highlighting tools developed for both individual users and businesses.

## Recent Announcements and News
- **OpenAI DevDay:** A new event aimed at bringing developers together to explore new tools and share ideas.
- **OpenAI o1:** Introduction of a series of AI models that emphasize thoughtfulness in responses.
- **Partnership with Apple:** Announcement of a collaboration to integrate ChatGPT into Apple experiences.
- **ChatGPT Enhancements:** New capabilities that allow ChatGPT to see, hear, and speak.
- **Structured Outputs in the API:** Introduction of new compliance and administrative tools for ChatGPT Enterprise, alongside improvements for data analysis products.

## Key Features
- **Product Offerings:** Includes ChatGPT for individuals, teams, and enterprise solutions with improved compliance tools.
- **SearchGPT Prototype:** A prototype featuring new AI search capabilities designed to enhance the search experience.
- **Improvements in AI Capabilities:** Ongoing enhancements to large language models for better reasoning and understanding.

Overall, OpenAI is committed to advancing AI technology while ensuring it aligns with safety and ethical standards.

In [56]:
display_summary("https://edwarddonner.com")

# Summary of Edward Donner's Website

Edward Donner's website serves as a platform for his interests in coding, experimenting with language models (LLMs), and creative pursuits like DJing and music production. He is the co-founder and CTO of Nebula.io, a company focused on utilizing AI to help individuals discover their potential in the talent management sector. His previous venture, untapt, was acquired in 2021. 

## Key Features:
- **Outsmart:** A project that involves a competitive arena for LLMs, emphasizing diplomacy and strategy.
- **Posts:** The website includes a blog with several recent posts about:
  - The Outsmart LLM Arena (August 6, 2024)
  - Resources for choosing the right LLM (June 26, 2024)
  - Fine-tuning LLMs on personal texts, including a simulation and techniques like QLoRA (February 7, 2024; January 31, 2024).

Donner encourages visitors to connect with him and engage in discussions related to AI and programming.

In [57]:
display_summary("https://cnn.com")

# Summary of CNN Website

The CNN website features a wide array of **breaking news, articles, and videos** from various categories, including **US and world news, politics, business, health, entertainment, sports, and science**. 

## Recent Major News Highlights:
- **Escalation in Israel-Lebanon Conflict**: Israel vows retaliation against Iran following missile attacks, leading to significant military activity and troop casualties in Lebanon.
- **Vice Presidential Debate Insights**: Analysis of a recent debate where JD Vance and Tim Walz presented their views, revealing no clear winner according to a CNN poll.
- **Global News Updates**: Coverage includes Thai school bus fire resulting in 23 fatalities, and other significant international events.
- **Health Advisories**: Recommendations have been made for flu and COVID-19 vaccinations as the season approaches.

## Other Notable Stories:
- **Upcoming Cultural Events**: Mention of fashion highlights from **Paris Fashion Week** and the resurgence of actors like **Daniel Day-Lewis** in new projects.
- **Weather and Climate Focus**: Reports on severe weather conditions affecting the US, particularly in the southeast due to recent storms.

Overall, the site encapsulates a mix of urgent news alerts and in-depth analyses, catering to a diverse audience seeking current affairs updates.